In [89]:
import numpy as np
from random import choice

In [90]:
class Restaurant:
    def __init__(self, mu, dev):
        self.mu = mu
        self.dev = dev
    def sample(self):
        return np.random.normal(self.mu, self.dev)

In [91]:
def explore_only(candidates, num_days):
    scores = []
    for _ in range(num_days):
        scores.append(choice(candidates).sample())
    return sum(scores)

In [92]:
def exploit_only(candidates, num_days):
    scores = [c.sample() for c in candidates]
    chosen = candidates[np.argmax(scores)]
    for _ in range(num_days - len(candidates)):
        scores.append(chosen.sample())
    return sum(scores)

In [93]:
def epsilon_greedy(candidates, num_days, epsilon=0.05):
    scores = []
    history = {idx: [c.sample()] for idx,c in enumerate(candidates)}
    for _ in range(num_days - len(candidates)):
        p = np.random.random()
        #explore
        if p < epsilon:
            chosen = choice(candidates)
        #exploit
        else:
            chosen = candidates[sorted(history.items(), key=lambda pair: np.mean(pair[1]))[-1][0]]
        score = chosen.sample()
        scores.append(score)
        history[candidates.index(chosen)].append(score)
    return sum(scores)

In [94]:
def ucb1(candidates, num_days):
    scores = []
    history = {idx: [c.sample()] for idx,c in enumerate(candidates)}
    for t in range(len(candidates), num_days):
        mu_plus_ucb = [np.mean(history[idx]) + np.sqrt(2*np.log(t) / len(history[idx])) for idx in range(len(candidates))]
        chosen = candidates[np.argmax(mu_plus_ucb)]
        
        score = chosen.sample()
        scores.append(score)
        history[candidates.index(chosen)].append(score)
    return sum(scores)

In [171]:
dev_factor = 0.5
num_restaurants = 3

mu_vals = [3*i for i in range(1,num_restaurants+1)]
dev_vals = [mu*dev_factor for mu in mu_vals]
mu_dev_pairs = zip(mu_vals, dev_vals)

candidates = [Restaurant(mu,dev) for mu,dev in mu_dev_pairs]

num_days = 300

optimal_average = max(mu_vals)*num_days

In [172]:
explore_only_vals = []
for _ in range(1000):
    val = explore_only(candidates, num_days)
    explore_only_vals.append(val)
print('Explore Only Mean Regret: %s'%((optimal_average - np.mean(explore_only_vals)) / optimal_average))

Explore Only Mean Regret: 0.33400345242040025


In [173]:
exploit_only_vals = []
for _ in range(1000):
    val = exploit_only(candidates, num_days)
    exploit_only_vals.append(val)
print('Exploit Only Mean Regret: %s'%((optimal_average - np.mean(exploit_only_vals)) / optimal_average))

Exploit Only Mean Regret: 0.10974979914722435


In [174]:
epsilon_greedy_vals = []
for _ in range(1000):
    val = epsilon_greedy(candidates, num_days, 0.1)
    epsilon_greedy_vals.append(val)
print('Epsilon Greedy Mean Regret (10%%): %s'%((optimal_average - np.mean(epsilon_greedy_vals)) / optimal_average))

Epsilon Greedy Mean Regret (10%): 0.061901290618584424


In [175]:
ucb1_vals = []
for _ in range(1000):
    val = ucb1(candidates, num_days)
    ucb1_vals.append(val)
print('UCB1 Mean Regret: %s'%((optimal_average - np.mean(ucb1_vals)) / optimal_average))

UCB1 Mean Regret: 0.05807450789812113
